## Activate Google Earth Engine Account

In [2]:
import ee
# Initialize the GEE API
try:
    ee.Initialize()
except Exception as ee:
    ee.Authenticate()
    ee.Initialize()

## Main toolbox

In [3]:
""" Import the libraries required by the program"""
#------------------------------------------------------------------------------------------------------
# geemap:A Python package for interactive mapping with Google Earth Engine, ipyleaflet, and ipywidgets
# Documentation: https://geemap.org
import geemap
# from geemap import ee_basemaps

import matplotlib.pyplot as plt

#eemont
import eemont

# import developed utilities
import Utilities as ut

# geetols: Google earth engine tools
# https://github.com/gee-community/gee_tools
import geetools
from geetools import tools

# hydrafloods: Hydrologic Remote Sensing Analysis for Floods
#https://github.com/Servir-Mekong/hydra-floods
import hydrafloods as hf
from hydrafloods import geeutils

# Ipywidgets for GUI design
import ipywidgets as ipw
from IPython.display import display
from ipywidgets import HBox, VBox, Layout

# A simple Python file chooser widget for use in Jupyter/IPython in conjunction with ipywidgets
# https://pypi.org/project/ipyfilechooser/
from ipyfilechooser import FileChooser

# Plotly Python API for interactive graphing
import plotly
import plotly.express as px
import plotly.graph_objects as go

# Pandas -  Python Data Analysis Library for data analysis and manipulation
import pandas as pd

# Miscellaneous Python modules
from datetime import datetime, timedelta
import os

#----------------------------------------------------------------------------------------------------



""" UI Design"""
#---------------------------------------------------------------------------------------------------
# Program Title
Title_text =  ipw.HTML(
    "<h3 class= 'text-center'><font color = 'blue'>Python-GEE Surface Water Analyzer Toolbox v.1.0.0</font>")
style = {'description_width': 'initial'}


# Image Processing Tab
#************************************************************************************************
# Image Parameters UI
dataset_description = ipw.Label('Satellite Imagery Parameters')
dataset_Label = ipw.Label('Select Dataset:', layout=Layout(margin='5px 0 0 5px')) #top right bottom left
Platform_options = ['Landsat', 'Sentinel-1', 'Sentinel-2', 'USDA NAIP' ]
Platform_dropdown = ipw.Dropdown(options = Platform_options, value = None,
                                   layout=Layout(width='150px', margin='5px 0 0 5px'))
PlatformType = HBox([dataset_Label, Platform_dropdown])

# Study period definition
#************************************************************************************************
# Start date picker
lbl_start_date = ipw.Label('Start Date:', layout=Layout(margin='5px 0 0 5px'))
start_date = ipw.DatePicker(value = datetime.now()-timedelta(7), disabled=False, 
                            layout=Layout(width='150px', margin='5px 0 0 30px'))
start_date_box = HBox([lbl_start_date, start_date])

# End date picker
lbl_end_date = ipw.Label('End Date:', layout=Layout(margin='5px 0 0 5px'))
end_date = ipw.DatePicker(value = datetime.now(), disabled=False, 
                          layout=Layout(width='150px', margin='5px 0 0 34px'))
end_date_box = HBox([lbl_end_date, end_date])

datePickers = VBox([start_date_box, end_date_box])


# Cloud threshold for filtering data
#************************************************************************************************
# Set cloud threshold
cloud_threshold = ipw.IntSlider(description = 'Cloud Threshold:',
                                orientation = 'horizontal',
                                 value = 50,
                                 step = 5,
                               style = style)

imageParameters = VBox([dataset_description, PlatformType, datePickers, cloud_threshold], 
                   layout=Layout(width='305px', border='solid 2px black'))


# Study Area definition
#************************************************************************************************
# Option to use a map drawn boundary or upload shapefile
StudyArea_description = ipw.Label('Study Area Definition')
user_preference = ipw.RadioButtons(options=['Map drawn boundary','Upload boundary'], value='Map drawn boundary')
file_selector = FileChooser(description = 'Upload', filter_pattern = ["*.shp","*.kml", "*.kmz"], use_dir_icons = True)

# Retrieve and process satellite images
#***********************************************************************************************
# Button to retrieve and process satellite images from the GEE platform
imageProcessing_Button = ipw.Button(description = 'Process images',
                                    tooltip='Click to process images', button_style = 'info',
                                   layout=Layout(width='150px', margin='5px 0 0 50px'))

# Study area UI and process button container
# ************************************************************************************************
StudyArea = VBox(children = [StudyArea_description, user_preference, imageProcessing_Button], 
                   layout=Layout(width='300px', border='solid 2px black', margin='0 0 0 10px'))



# Results UI for displaying number and list of files
#*****************************************************************************************************
lbl_results = ipw.Label('Processing Results')
lbl_images = ipw.Label('No. of processed images:')
lbl_RetrievedImages = ipw.Label()
display_no_images = HBox([lbl_images, lbl_RetrievedImages])
lbl_files = ipw.Label('List of files:')
lst_files = ipw.Select(layout=Layout(width='360px', height='100px'))

image_Results = VBox([lbl_results, display_no_images, lbl_files, lst_files], 
                   layout=Layout(width='400px', border='solid 2px black', margin='0 0 0 10px'))


# Container for Image Processing Tab
#************************************************************************************************
imageProcessing_tab = HBox([imageParameters, StudyArea, image_Results])


# Water Extraction Tab
#*************************************************************************************************
# Water extraction indices
water_index_options = ['NDWI','MNDWI','DSWE', 'AWEInsh', 'AWEIsh']
lbl_indices = ipw.Label('Water Index:', layout=Layout(margin='5px 0 0 5px'))
water_indices = ipw.Dropdown(options = water_index_options,
                             value = 'NDWI',
                            layout=Layout(width='100px', margin='5px 0 0 63px'))
display_indices = HBox([lbl_indices, water_indices])

# Color widget for representing water
lbl_color = ipw.Label('Color:', layout=Layout(margin='5px 0 0 5px'))
index_color =  ipw.ColorPicker(concise = False,
                              value = 'blue',
                              layout=Layout(width='100px', margin='5px 0 0 101px'))
display_color_widget = HBox([lbl_color, index_color])

# Water index threshold selection
threshold_options = ['Simple','Otsu','Bmax_Otsu', 'Edge_Otsu']

lbl_threshold_method = ipw.Label('Thresholding Method:', layout=Layout(margin='5px 0 0 5px'))
threshold_dropdown = ipw.Dropdown(options = threshold_options,
                             value = 'Simple',
                            layout=Layout(width='100px', margin='5px 0 0 10px'))
display_thresholds = HBox([lbl_threshold_method, threshold_dropdown])


lbl_threshold = ipw.Label('Threshold value:', layout=Layout(margin='5px 0 5px 5px'))
threshold_value = ipw.BoundedFloatText(value=0.000, min = -1.0, max = 1.0, step = 0.050,
                                       layout=Layout(width='100px', margin='5px 0 0 40px'))

display_threshold_widget = HBox([lbl_threshold, threshold_value])

water_index_Box = VBox([display_indices, display_color_widget, display_thresholds, display_threshold_widget],
                      layout=Layout(width='250px', border='solid 2px black'))

extractWater_Button = ipw.Button(description = 'Extract Water',
                                tooltip='Click to extract surface water', 
                                button_style = 'info',
                                layout=Layout(width='150px', margin='5px 0 0 20px'))

Extraction_tab = HBox([water_index_Box, extractWater_Button])


# Spatial Analysis Tab
#**************************************************************************************************

water_Frequency_button = ipw.Button(description = 'Compute Water Frequency',
                                    tooltip='Click to compute water occurence frequency',
                                    button_style = 'info', layout=Layout(width='200px'))

Depths_Button = ipw.Button(description = 'Depth Map',
                                tooltip='Click to generate depth maps', button_style = 'info',
                                layout=Layout(width='200px'))

elevData_options = ipw.Dropdown(options=['NED','SRTM'], value='NED', description='Elevation Dataset:',
                                layout=Layout(width='200px', margin='5px 0 0 10px'), style = style)
lbl_Elev = ipw.Label('Elevation Dataset:', layout=Layout(margin='5px 0 5px 5px'))

elev_Box = HBox([Depths_Button, elevData_options])

removeCloud_Button = ipw.Button(description = 'Remove Clouds',
                                tooltip='Click to remove clouds', button_style = 'info',
                               layout=Layout(width='200px'))

Spatial_Analysis_Tab = VBox([water_Frequency_button, elev_Box, removeCloud_Button])


# Ploting and Statistics Tab
#***************************************************************************************************
area_unit = ipw.Dropdown(options = ['Square Km', 'Hectares', 'Acre'],
                             value = 'Square Km',
                            description = 'Unit for water surface area:',
                            style=style,
                            tooltip='Select unit for areas')

plot_button = ipw.Button(description = 'Plot',
                        tooltip='Click to plot graph', button_style = 'info',
                        layout=Layout(width='100px', border='solid 2px black',margin='5 0 0 50px'))
plotting_box = VBox([area_unit, plot_button], layout=Layout(width='310px', border='solid 2px black'))

file_selector1 = FileChooser(description = 'Select folder and filename', filter_pattern = "*.csv", use_dir_icons = True)
file_selector1.title = '<b>Select Folder and Filename</b>'
file_selector1.default_path = os.getcwd()

save_data_button = ipw.Button(description = 'Save Data',
                            tooltip='Click to save computed areas to file', button_style = 'info')


save_box = VBox(children=[file_selector1, save_data_button],
               layout=Layout(width='550px', border='solid 2px black', margin='0 0 0 10px'))
plot_stats_tab = HBox(children=[plotting_box, save_box])

# Downloads Tab
#***************************************************************************************************
files_to_download = ipw.RadioButtons(options=['Satellite Images', 'Water Mask', 'Water Frequency', 'DSWE Images'], 
                                     value='Satellite Images', description='Files to download:', style = style)

download_location = ipw.RadioButtons(options=['Google Drive', 'Local Disk'], 
                                     value='Google Drive', description='Download Location:', style = style)

folder_name = ipw.Text(description='Folder Name:')

folder_selector = FileChooser(description = 'Select Folder', show_only_dirs = True, use_dir_icons = True)
folder_selector.title = '<b>Select a folder</b>'
folder_selector.default_path = os.getcwd()

download_button = ipw.Button(description = 'Download',
                                    tooltip='Click to plot download water images', button_style = 'info')


download_settings = VBox(children=[files_to_download, download_location, folder_name])

download_tab = HBox([download_settings, download_button])

# Functions to control UI changes and parameter settings
#****************************************************************************************************
def platform_index_change(change):
    """
    Function to set image visualization parameters, hide or show some UI componets and
    show water indices that are applicable to the type of satellite image selected
        
    args:
        None

    returns:
        None
    """
    try:
        global img_type
        global visParams
        global water_index_options
        if Platform_dropdown.value == 'Landsat':
            visParams = {'bands': ['red', 'green', 'blue'],
                  'min': 0,
                  'max': 3000,
                  'gamma': [0.95, 1.1, 1]}
            cloud_threshold.disabled = False
            water_indices.disabled = False
            index_color.disabled = False
            threshold_value.disabled = False
            water_indices.options = ['NDWI','MNDWI','DSWE','AWEInsh', 'AWEIsh']
            threshold_dropdown.options = ['Simple','Otsu','Bmax_Otsu', 'Edge_Otsu']
        elif Platform_dropdown.value == 'Sentinel-1':
            visParams = {'min': -25,'max': 5}
            cloud_threshold.disabled = True
            water_indices.disabled = True
            index_color.disabled = True
            threshold_value.disabled = True
            threshold_dropdown.options = ['Otsu','Bmax_Otsu', 'Edge_Otsu']
        elif Platform_dropdown.value == 'Sentinel-2':
            visParams = {'bands': ['red', 'green', 'blue'],
              'min': 0.0,
              'max': 3000}
            cloud_threshold.disabled = False
            water_indices.disabled = False
            index_color.disabled = False
            threshold_value.disabled = False
            water_indices.options = ['NDWI','MNDWI','AWEInsh', 'AWEIsh']
            threshold_dropdown.options = ['Simple','Otsu','Bmax_Otsu', 'Edge_Otsu']
        elif Platform_dropdown.value == 'USDA NAIP':
            visParams = {'bands': ['R', 'G','B'],
                        'min': 0.0,
                        'max': 255.0}
            threshold_value.disabled = False
            water_indices.disabled = False
            index_color.disabled = False
            water_indices.options = ['NDWI']
            threshold_dropdown.options = ['Simple','Otsu','Bmax_Otsu', 'Edge_Otsu']
    except Exception as e:
             print(e)
# Link widget to function
Platform_dropdown.observe(platform_index_change, 'value')

def showFileSelector(button):
    """
    Function to show or hide shapefile upload widget
        
    args:
        None

    returns:
        None
    """
    if button['new']:
        StudyArea.children = [StudyArea_description, user_preference, file_selector, imageProcessing_Button]
    else:
        StudyArea.children = [StudyArea_description, user_preference,imageProcessing_Button]

# Link widget to file selector function
user_preference.observe(showFileSelector, names='index')

def showLocationSelector(button):
    """
    Function to show or hide folder selector
        
    args:
        None

    returns:
        None
    """
    if button['new']:
        download_settings.children = [files_to_download, download_location, folder_selector]
    else:
        download_settings.children = [files_to_download, download_location, folder_name]
        
# Link widget to folder selector function
download_location.observe(showLocationSelector, names='index')

#****************************************************************************************************

# Full UI
#***************************************************************************************************
tab_children = [imageProcessing_tab, Extraction_tab, Spatial_Analysis_Tab, plot_stats_tab, download_tab]

tab = ipw.Tab()
tab.children = tab_children

# changing the title of the first and second window
tab.set_title(0, 'Image Processing')
tab.set_title(1, 'Water Extraction')
tab.set_title(2, 'Spatial Analysis')
tab.set_title(3, 'Plotting & Stats')
tab.set_title(4, 'Download Images')


# Plotting outputs and feedback to user
#***************************************************************************************************
plot_output = ipw.Output()
feedback = ipw.Output()
OUTPUTS = VBox([plot_output, feedback])

# create map instance
Map =  geemap.Map()
Map.add_basemap('HYBRID')

GUI = VBox([Title_text,tab,Map])

def reset_map(self):
    Map.clear_layers()
    Map.remove_control(Map.colorbar)
    Map.add_basemap('HYBRID')

# Function to clip images
def clipImages(img):
    """
    Function to clip images

    args:
        Image

    returns:
        Clipped image
    """
    orig = img
    clipped_image = img.clip(site).copyProperties(orig, orig.propertyNames())
    return clipped_image    


def process_images(self):
    """
    Function to retrieve and process satellite images from GEE platform
        
    args:
        None

    returns:
        None
    """
    with plot_output:
        plot_output.clear_output()
        try:
            global filtered_Collection
            global filtered_landsat
            global clipped_images
            global imageType
            global dates
            global site
            global img_scale
            global file_list
            global StartDate
            global EndDate
            
            # reset the map
#             Map.clear_layers()
#             Map.add_basemap('HYBRID')
#             Map.remove_control(Map.colorbar)
            
            lbl_RetrievedImages.value = 'Processing....'
            cloud_thresh = cloud_threshold.value

            # Define study area based on user preference
            if user_preference.index == 1:
                file = file_selector.selected  
                site = ut.load_boundary(file)
                Map.center_object(site, 15)
            else:
                site = ee.FeatureCollection(Map.draw_last_feature)

            # get widget values
            imageType = Platform_dropdown.value
            StartDate = ee.Date.fromYMD(start_date.value.year,start_date.value.month,start_date.value.day)
            EndDate = ee.Date.fromYMD(end_date.value.year,end_date.value.month,end_date.value.day)

            # filter image collection based on date, study area and cloud threshold(depends of datatype)
            if imageType == 'Landsat':
                filtered_landsat = ut.load_Landsat(site, StartDate, EndDate, cloud_thresh)
                filtered_Collection = filtered_landsat.map(ut.maskLandsatclouds)
            elif imageType == 'Sentinel-2':
                Collection_before = ut.load_Sentinel2(site, StartDate, EndDate, cloud_thresh)
                filtered_Collection = Collection_before.map(ut.maskS2clouds)
            elif imageType == 'Sentinel-1':
                Collection_before = ut.load_Sentinel1(site, StartDate, EndDate)
                # apply speckle filter algorithm or smoothing
                filtered_Collection = Collection_before.map(hf.lee_sigma)
            elif imageType == 'USDA NAIP':
                filtered_Collection = ut.load_NAIP(site, StartDate, EndDate)

            # Clip images to study area
            clipped_images = filtered_Collection.map(clipImages)
            
            # Mosaic same day images
            clipped_images = tools.imagecollection.mosaicSameDay(clipped_images)

            # Add first image in collection to Map
            first_image = clipped_images.first()
            if imageType == 'Sentinel-1':
                img_scale = first_image.select(0).projection().nominalScale().getInfo()
            else:
                bandNames = first_image.bandNames().getInfo()
                img_scale = first_image.select(str(bandNames[0])).projection().nominalScale().getInfo()

            Map.addLayer(site, {}, 'AOI')
            Map.addLayer(clipped_images, visParams, imageType)
            
            # Get no. of processed images
            no_of_images = filtered_Collection.size().getInfo()

            # Display number of images
            lbl_RetrievedImages.value = str(no_of_images)

            # List of files
            file_list = filtered_Collection.aggregate_array('system:id').getInfo()
            # display list of files
            lst_files.options = file_list

        except Exception as e:
                 print(e)
                 print('An error occurred during processing.')
                    
# Return the DN that maximizes interclass variance in the region        
def otsu(histogram):
    """
    Function to use Otsu algorithm to compute DN that maximizes interclass variance in the region 

    args:
        Histogram

    returns:
        Otsu's threshold
    """
    counts = ee.Array(ee.Dictionary(histogram).get('histogram'))
    means = ee.Array(ee.Dictionary(histogram).get('bucketMeans'))
    size = means.length().get([0])
    total = counts.reduce(ee.Reducer.sum(), [0]).get([0])
    sum = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0])
    mean = sum.divide(total)
    indices = ee.List.sequence(1, size)
    
    # Compute between sum of squares, where each mean partitions the data.
    def func_bss(i):
        aCounts = counts.slice(0, 0, i)
        aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
        aMeans = means.slice(0, 0, i)
        aMean = aMeans.multiply(aCounts) \
            .reduce(ee.Reducer.sum(), [0]).get([0]) \
            .divide(aCount)
        bCount = total.subtract(aCount)
        bMean = sum.subtract(aCount.multiply(aMean)).divide(bCount)
        return aCount.multiply(aMean.subtract(mean).pow(2)).add(
               bCount.multiply(bMean.subtract(mean).pow(2)))
    
    bss = indices.map(func_bss)
    return means.sort(bss).get([-1])


def Water_Extraction(self):
    """
    Function to extract surface water from satellite images

    args:
        None

    returns:
        None
    """
    try:
        global water_images
        global dswe_images
        global index_image

        color_palette = index_color.value       
        # Function to extract water using NDWI or MNDWI from multispectral images
        def extract_water(img):
            """
            Function to extract surface water from Landsat and Sentinel-2 images using
            water extraction indices: NDWI, MNDWI, and AWEI

            args:
                Image

            returns:
                Image with water mask
            """
            index_image = ee.Image(1)
            if water_indices.value == 'NDWI':
                if imageType == 'Landsat' or imageType == 'Sentinel-2':
                    bands = ['green', 'nir']
                elif imageType == 'USDA NAIP':
                    bands = ['G', 'N']
                index_image = img.normalizedDifference(bands).rename('waterMask')
            elif water_indices.value == 'MNDWI':
                bands = ['green', 'swir1']
                index_image = img.normalizedDifference(bands).rename('waterMask')
            elif water_indices.value == 'AWEInsh':
                index_image = img.expression(
                        '(4 * (GREEN - SWIR1)) - ((0.25 * NIR)+(2.75 * SWIR2))', {
                            'NIR': img.select('nir'),
                            'GREEN': img.select('green'),
                            'SWIR1': img.select('swir1'),
                            'SWIR2': img.select('swir2')
                        }).rename('waterMask')
            elif water_indices.value == 'AWEIsh':
                index_image = img.expression(
                        '(BLUE + (2.5 * GREEN) - (1.5 * (NIR + SWIR1)) - (0.25 * SWIR2))', {
                            'BLUE':img.select('blue'),
                            'NIR': img.select('nir'),
                            'GREEN': img.select('green'),
                            'SWIR1': img.select('swir1'),
                            'SWIR2': img.select('swir2')
                        }).rename('waterMask')

            # Compute threshold
            if threshold_dropdown.value == 'Simple': # Simple value no dynamic thresholding
                nd_threshold = threshold_value.value
            elif threshold_dropdown.value == 'Otsu':
#                 Compute histogram
                reducers = ee.Reducer.histogram(255,2).combine(reducer2=ee.Reducer.mean(), sharedInputs=True)\
                    .combine(reducer2=ee.Reducer.variance(), sharedInputs= True)
                histogram = index_image.select('waterMask').reduceRegion(
                reducer=reducers,
                geometry=site.geometry(),
                scale=img_scale,
                bestEffort=True)
                nd_threshold = otsu(histogram.get('waterMask_histogram'))
            elif threshold_dropdown.value == 'Bmax_Otsu':
                pass
            elif threshold_dropdown.value == 'Edge_Otsu':
                pass
                
            water_image = index_image.gt(nd_threshold).selfMask().copyProperties(img, ['system:time_start'])    
            return water_image
        
        # Function to extract water from SAR Sentinel 1 images
        def add_S1_waterMask(img):
            """
            Function to extract surface water from Sentinel-1 images Otsu algorithm

            args:
                Image

            returns:
                Image with water mask
            """
            # Compute histogram
            reducers = ee.Reducer.histogram(255,2).combine(reducer2=ee.Reducer.mean(), sharedInputs=True)\
                .combine(reducer2=ee.Reducer.variance(), sharedInputs= True)
            histogram = img.select('VV').reduceRegion(
            reducer=reducers,
            geometry=site.geometry(),
            scale=img_scale,
            bestEffort=True)

            # Calculate threshold via function otsu (see before)
            threshold = otsu(histogram.get('VV_histogram'))
            
            # get watermask
            waterMask = img.select('VV').lt(threshold).rename('waterMask')
            waterMask = waterMask.updateMask(waterMask) #Remove all pixels equal to 0
            return img.addBands(waterMask)
        
        def maskDSWE_Water(img):
            waterImage = img.select('dswe').rename('waterMask')
            watermask = waterImage.gt(0).And(waterImage.lt(5)).selfMask().copyProperties(img, ['system:time_start'])
            return watermask
    
        if imageType == 'Sentinel-1':
            water_images = clipped_images.map(add_S1_waterMask).select('waterMask')
            visParams = {'min': 0,'max': 1, 'palette': color_palette}
            Map.addLayer(water_images.first(), visParams, 'Water')
        elif imageType == 'Landsat':
            if water_indices.value == 'DSWE':
                dem = ee.Image('USGS/NED')
                dswe_images = ut.DSWE(filtered_landsat, dem, site)
                 # Viz parameters: classes: 0, 1, 2, 3, 4, 9
                dswe_viz = {'min':0, 'max': 9, 'palette': ['000000', '002ba1', '6287ec', '77b800', 'c1bdb6', '000000', 
                                                           '000000', '000000', '000000', 'ffffff']}
                water_images = dswe_images.map(maskDSWE_Water)
                Map.addLayer(dswe_images, dswe_viz, 'DSWE')
            else:
                water_images = clipped_images.map(extract_water)
            Map.addLayer(water_images.first(), {'palette': color_palette}, 'Water')
            
        else:
            water_images = clipped_images.map(extract_water)
            Map.addLayer(water_images.first(), {'palette': color_palette}, 'Water')

        #ndwi_palette = ['#ece7f2', '#d0d1e6', '#a6bddb', '#74a9cf', '#3690c0', '#0570b0', '#045a8d', '#023858']
        
    except Exception as e:
             print(e)
             print('An error occurred during computation.')

threshold_value.observe(Water_Extraction, 'value')
    
def Sharpen_Image(self):
    """
    Function to pansharpen Landsat and Sentinel-2 images
    Sentinel-2 image pansharpening algorithm yet to be implemented

    args:
        None

    returns:
        None
    """
    try: 
        # function to sharpen Landsat images
        def sharpen_Landsat(img):
            """
            Function to pansharpen Landsat images
            
            args:
                Image

            returns:
                Sharpened image
            """
            hsv = img.select(['B4', 'B3', 'B2']).rgbToHsv()
            sharpened_img = ee.Image.cat([hsv.select('hue'), hsv.select('saturation'), img.select('B8')]).hsvToRgb()
            return sharpened_img
        
        if imageType == 'Landsat':
            visParams = {
              'min': 0,
              'max': 0.25,
              'gamma': [1.1, 1.1, 1.1]}
            sharpened_images = clipped_images.map(sharpen_Landsat)
            Map.addLayer(sharpened_images, visParams, 'L8 Sharpened')
        else:
            # function to sharpen Sentinel-2 images yet to be added
            pass
              
    except Exception as e:
             print(e)
             print('An error occurred during computation.')
    
def maskclouds(self):
    """
    Function to mask clouds from Landsat and Sentinel-2 images
    
    args:
        None

    returns:
        None
    """
    try: 
        
        if imageType == 'Landsat':
            cloudRemovedImages = clipped_images.map(ut.maskLandsatclouds)
            #cloudRemovedImages = clipped_images.map(cloud_mask.landsatSR())
            # Visualization for TOA
            visParams = {'bands': ['B4', 'B3', 'B2'],
                  'min': 0,
                  'max': 0.25,
                  'gamma': [1.1, 1.1, 1]}
            Map.addLayer(cloudRemovedImages, visParams, 'L8 Cloud Masked')
            
        elif imageType == 'Sentinel-2':
            cloudRemovedImages = clipped_images.map(ut.maskS2clouds)
            visParams = {'bands': ['B4', 'B3', 'B2'],
              'min': 0.0,
              'max': 3000}
            Map.addLayer(cloudRemovedImages, visParams, 'S2 Cloud Masked')
        else:
            pass
            
    except Exception as e:
             print(e)
             print('An error occurred during computation.')
                
def calc_area(img):
    """
    Function to calculate area of water pixels

    args:
        Water mask image

    returns:
        Water image with calculated total area of water pixels
    """
    global unit_symbol
    unit = area_unit.value
    divisor = 1
    if unit =='Square Km':
        divisor = 1e6
        unit_symbol = 'Sqkm'
    elif unit =='Hectares':
        divisor = 1e4
        unit_symbol = 'Ha'
    else:
        divisor = 4047
        unit_symbol = 'acre'
        
    pixel_area = img.multiply(ee.Image.pixelArea()).divide(divisor)
    img_area = pixel_area.reduceRegion(**{
        'geometry': site.geometry(),
        'reducer': ee.Reducer.sum(),
        'scale': img_scale,
        'maxPixels': 1e13
    })
    return img.set({'water_area': img_area})

def update_point(trace, points, selector):
    date = df['Date'].iloc[points.point_inds].values[0]
    date = pd.to_datetime(str(date))
    date = ee.Date(date)
    date = date.millis()
    equalDate = ee.Filter.equals('system:time_start', date)
    wImage = ee.Image(water_images.filter(equalDate).first())
    Map.addLayer(wImage, {'palette': color_palette}, 'Water_1')


def plot_areas(self):
    """
    Function to plot a time series of calculated water area for each water image
    
    args:
        None

    returns:
        None
    """
    try:
        global df
        # Compute water areas
        water_areas = water_images.map(calc_area)
        water_stats = water_areas.aggregate_array('water_area').getInfo()

        dates = water_images.aggregate_array('system:time_start')\
            .map(lambda d: ee.Date(d).format('YYYY-MM-dd')).getInfo()
        
        with plot_output:
            plot_output.clear_output()
            dates_lst = [datetime.strptime(i, '%Y-%m-%d') for i in dates]
            y = [item.get('waterMask') for item in water_stats]
            df = pd.DataFrame(list(zip(dates_lst,y)), columns=['Date','Area'])
            
#             plt.ioff()
#             fig = plt.figure()
#             plt.ion()
#             fig.canvas.toolbar_visible = True
#             plt.plot(df['Date'],df['Area'],color='blue', marker='o',ls='-', label=r'$Observed$')
#             plt.xlabel('Date')
#             plt.ylabel('Area (${}$)'.format(unit_symbol))
#             fig.show()
        
            fig = go.Figure([go.Scatter(x=df['Date'], y=df['Area'], name='Water Hydrograph', 
                    mode='lines+markers', line=dict(dash = 'solid', color ='Black', width = 0.5))])

            # plotly figure layout
            fig.update_layout(title = '<b>Surface Water Area Hydrograph<b>', 
                              title_x = 0.5, title_y = 0.90, title_font=dict(family="Arial",size=24),
                              template = "plotly_white",
                              xaxis =dict(title ='<b>Date<b>', linecolor = 'Black'),
                              yaxis=dict(title='Area ('+unit_symbol+')', linecolor = 'Black'),
                              font_family="Arial")

            fig.show()
           
            
    except Exception as e:
            print(e)
            print('An error occurred during computation.')

def save_data(self):
    filename = file_selector1.selected
    df.to_csv(filename, index=False)

def dowload_images(self):
        with feedback:
            try:
                path = folder_selector.selected_path
                folder = folder_name.value
                name_Pattern = '{sat}_{system_date}_{imgType}'
                date_pattern = 'ddMMMy'
                extra = dict(sat=imageType, imgType = 'Water')
                if files_to_download.index == 0:
                    download_images = clipped_images
                    extra = dict(sat=imageType, imgType = 'Satellite')
                elif files_to_download.index == 1:
                    download_images = water_images
                    extra = dict(sat=imageType, imgType = 'Water')
                elif files_to_download.index == 2:
                    download_images = ee.ImageCollection([water_frequency])
                    name_Pattern = '{sat}_{start}_{end}_{imgType}'
                    extra = dict(sat=imageType, imgType = 'Frequency', start=start_date.value.strftime("%x"),
                                 end=end_date.value.strftime("%x"))
                else:
                    download_images = dswe_images
                    extra = dict(sat=imageType, imgType = 'DSWE')
                
                if download_location.index == 0:
                    task = geetools.batch.Export.imagecollection.toDrive(
                        collection = download_images,
                        folder = folder,
                        region = site.geometry(),
                        namePattern = name_Pattern,
                        scale = img_scale,
                        datePattern=date_pattern,
                        extra = extra,
                        verbose=True,
                        maxPixels = int(1e13))
                    task
                else:
                    print("Downloading files...")
                    geemap.ee_export_image_collection(download_images, out_dir=path)

                feedback.clear_output()
                print("Download complete")

            except Exception as e:
                    print(e)
                    print('Download could not be completed')
                    
def water_frequency(self):
    with feedback:
        try:
            global water_frequency
            water_occurence =  water_images.reduce(ee.Reducer.sum())
            water_frequency = water_occurence.divide(water_images.size()).multiply(100)
            visParams = {'min':0, 'max':100, 'palette': ['orange','yellow','blue','darkblue']}
            Map.addLayer(water_frequency, visParams, 'Water Frequency')
            
            colors = visParams['palette']
            vmin = visParams['min']
            vmax = visParams['max']
            
            Map.add_colorbar_branca(colors=colors, vmin=vmin, vmax=vmax, layer_name="Water Frequency")
        except Exception as e:
                    print(e)
                    print('Frequency computation could not be completed')
    
def get_dates(col):
    dates = ee.List(col.toList(col.size()).map(lambda img: ee.Image(img).date().format()))
    return dates

def calc_depths(self):
#     global depth_maps
#     if elevData_options.value =='NED':
#         demSource = 'USGS/NED'
#     else:
#         demSource = 'USGS/SRTMGL1_003'

#     # dem = ee.Image("users/collinsowusu09/GDM").select(['b1']).clip(site)
#     dem = ee.Image(demSource).select('elevation').clip(site)
#     projection = dem.projection()
#     resolution = projection.nominalScale().getInfo()
    
#     def estimate_depth(img):
#         flood = img
#         flood_image = flood.multiply(0)

#         # Outlier detection and filling on complete DEM using the modified z-score and a median filter [5]
#         kernel = ee.Kernel.fixed(3,3,[[1,1,1],[1,1,1],[1,1,1]])
#         kernel_weighted = ee.Kernel.fixed(3,3,[[1,1,1],[1,0,1],[1,1,1]])
#         median = dem.focal_median(**{'kernel':kernel}).reproject(projection)
#         median_weighted = dem.focal_median(**{'kernel':kernel_weighted}).reproject(projection)
#         diff = dem.subtract(median)
#         mzscore = diff.multiply(0.6745).divide(diff.abs().focal_median(**{'kernel':kernel}).reproject(projection))
#         fillDEM = dem.where(mzscore.gt(3.5),median_weighted)

#         # Outlier detection and filling on the flood extent border pixels
#         expand = flood_image.focal_max(**{'kernel': ee.Kernel.square(**{
#             'radius': projection.nominalScale(),
#             'units': 'meters'
#           })}).reproject(projection)
#         demMask = fillDEM.updateMask(flood_image.mask().eq(0))
#         boundary = demMask.add(expand)
#         medianBoundary = boundary.focal_median(**{'kernel':kernel}).reproject(projection)
#         medianWeightedBoundary = boundary.focal_median(**{'kernel':kernel_weighted}).reproject(projection)
#         diffBoundary = boundary.subtract(medianBoundary)
#         mzscoreBoundary = diffBoundary.multiply(0.6745).divide(diffBoundary.abs()\
#                             .focal_median(**{'kernel':kernel}).reproject(projection))
#         fill = fillDEM.where(mzscoreBoundary.gt(3.5),medianWeightedBoundary)
# #         fill = dem
#         # cumulativeCost floodwater surface elevation model (adaptation of the cost allocation method from FwDETv2.0)
#         mod = fill.updateMask(flood_image.mask().eq(0))
#         source = mod.mask()
#         val = 10000
#         push = 5000
#         cost0 = ee.Image(val).where(source,0).cumulativeCost(source,push)
#         cost1 = ee.Image(val).where(source,1).cumulativeCost(source,push)
#         cost2 = mod.unmask(val).cumulativeCost(source,push)
#         costFill = cost2.subtract(cost0).divide(cost1.subtract(cost0))
#         costSurface = mod.unmask(0).add(costFill)
#         # Interpolation method courtesy of Matt Hancher (Earth Engine Co-Founder) posted to the GEE developer forums
#         # https:#groups.google.com/forum/#!forum/google-earth-engine-developers

#         # Kernel size for low-pass filter
#         boxcar = ee.Kernel.square(**{'radius': 3, 'units': 'pixels', 'normalize': True})

#         # Floodwater depth calculation and smoothing using a low-pass filter
#         costDepth = costSurface.subtract(fill) \
#                           .rename('Depth') \
#                           .convolve(boxcar) \
#                           .reproject(projection) \
#                           .updateMask(flood_image.eq(0))

#         costDepthFilter = costDepth.where(costDepth.lt(0),0)
#         return costDepthFilter.copyProperties(flood, flood.propertyNames())
    
#     depth_maps = water_images.map(estimate_depth)
    
#     maximum_flood = ee.ImageCollection(water_images.max())
#     maximum_Depth = maximum_flood.map(estimate_depth)
    
#     minVal = ut.image_min_value(maximum_Depth.first(), site)
#     maxVal = ut.image_max_value(maximum_Depth.first(), site)

#     visParams = {'min':round(minVal), 'max':round(maxVal), 'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

#     Map.addLayer(maximum_Depth, visParams, 'Depth')
#     colors = visParams['palette']
#     Map.add_colorbar_branca(colors=colors, vmin=round(minVal,2), vmax=round(maxVal,2), layer_name='Depth')
    pass

imageProcessing_Button.on_click(process_images)
extractWater_Button.on_click(Water_Extraction)
Depths_Button.on_click(calc_depths)
removeCloud_Button.on_click(maskclouds)
plot_button.on_click(plot_areas)
download_button.on_click(dowload_images)
water_Frequency_button.on_click(water_frequency)
save_data_button.on_click(save_data)

display(GUI)
plot_output

Output()